# Sample for KFServing SDK 

This is a sample for KFServing SDK. 

The notebook shows how to use KFServing SDK to create, get, patch and delete KFService.

In [ ]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2ModelSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2KFServiceSpec
from kfserving import V1alpha2KFService

## Define KFService

Firstly define the model spec for default model spec, and then define the kfservice basic on the model spec.

In [ ]:
default_model_spec = V1alpha2ModelSpec(tensorflow=V1alpha2TensorflowSpec(
    storage_uri='gs://kfserving-samples/models/tensorflow/flowers'))
    
kfsvc = V1alpha2KFService(api_version=constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_model_spec))

## Create KFService

Call KFServingClient to create KFService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(kfsvc)

## Check the KFService

In [ ]:
KFServing.get('flower-sample', namespace='kubeflow')

## Patch the KFService 

Patch the KFService to add canary model.

In [ ]:
canary_model_spec = V1alpha2ModelSpec(tensorflow=V1alpha2TensorflowSpec(
    storage_uri='gs://kfserving-samples/models/tensorflow/flowers'))

kfsvc = V1alpha2KFService(api_version=constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_model_spec,
                                                     canary=canary_model_spec,
                                                     canary_traffic_percent=10))

KFServing.patch('flower-sample', kfsvc)

## Delete the KFService

In [ ]:
KFServing.delete('flower-sample', namespace='kubeflow')